In [1]:
%load_ext autoreload
%autoreload 2
import saopaulo.load_trips as sptr
import saopaulo.maps_aux as aux
import saopaulo.cycling_infrastructure as cinfra
import saopaulo.choropleth as spchoro
#import saopaulo.choropleth_folium as spchoro

import saopaulo.sp_grid as gr

from bikescience.intersect_ways import geometry_intersection_length

import geopandas as gpd
import json
import pandas as pd
import numpy as np
from ipywidgets import interact_manual, widgets, fixed
from IPython.core.display import display, HTML, clear_output
import folium

import warnings
import requests
import os
import fnmatch as fnm
warnings.simplefilter('ignore')

gr.SP_LAT = -23.63
gr.SP_LON = -46.55


/home/pedro/.local/lib/python3.8/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
distritos = [
    'Bela Vista', 'Bom Retiro', 'Cambuci', 'Consolação', 'Liberdade', 
    'República', 'Santa Cecília', 'Sé','Butantã', 'Morumbi', 'Raposo Tavares', 
    'Rio Pequeno', 'Vila Sônia', 'Barra Funda', 'Jaguara', 'Jaguaré', 'Lapa', 
    'Perdizes', 'Vila Leopoldina', 'Alto de Pinheiros', 'Itaim Bibi', 
    'Jardim Paulista', 'Pinheiros','Aricanduva', 'Carrão', 'Vila Formosa',
    'Água Rasa', 'Belém', 'Brás', 'Mooca', 'Pari', 'Tatuapé', 'Artur Alvim',
    'Cangaíba', 'Penha', 'Vila Matilde', 'São Lucas', 'Sapopemba',
    'Vila Prudente','Cidade Tiradentes', 'Ermelino Matarazzo', 'Ponte Rasa',
    'Guaianases', 'Lajeado', 'Itaim Paulista', 'Vila Curuçá', 'Cidade Líder',
    'Itaquera', 'José Bonifácio', 'Parque do Carmo', 'Iguatemi', 'São Mateus',
    'São Rafael', 'Jardim Helena', 'São Miguel', 'Vila Jacuí','Cachoeirinha', 
    'Casa Verde', 'Limão', 'Brasilândia', 'Freguesia do Ó', 'Anhanguera', 
    'Perus', 'Jaraguá', 'Pirituba', 'São Domingos','Jaçanã', 'Tremembé', 
    'Mandaqui', 'Santana', 'Tucuruvi', 'Vila Guilherme', 'Vila Maria', 
    'Vila Medeiros','Cursino', 'Ipiranga', 'Sacomã', 'Jabaquara', 'Moema', 
    'Saúde', 'Vila Mariana', 'Campo Limpo', 'Capão Redondo', 'Vila Andrade', 
    'Cidade Dutra', 'Grajaú', 'Socorro', 'Cidade Ademar', 'Pedreira', 
    'Jardim Ângela', 'Jardim São Luís', 'Marsilac', 'Parelheiros',
    'Campo Belo', 'Campo Grande', 'Santo Amaro']

In [3]:
zone_shp = gpd.read_file('../data/sao-paulo/od/shapes/Zonas_2017_region.shp')
display(zone_shp.loc[zone_shp['NomeDistri'] == 'Butantã'])

,NumeroZona,NomeZona,NumeroMuni,NomeMunici,NumDistrit,NomeDistri,Area_ha_2,geometry
338,339,Cidade Universitária,36,São Paulo,12,Butantã,584.56,"POLYGON Z ((321979.631 7393581.866 0.000, 3219..."
339,340,Butantã,36,São Paulo,12,Butantã,207.11,"POLYGON Z ((324176.344 7392045.849 0.000, 3241..."
340,341,Jardim Caxingui,36,São Paulo,12,Butantã,180.08,"POLYGON Z ((323783.329 7391868.109 0.000, 3239..."
341,342,Jardim Bonfiglioli,36,São Paulo,12,Butantã,319.97,"POLYGON Z ((322930.124 7390110.600 0.000, 3227..."


In [3]:
# loading data

# SIRGAS 2000 / UTM zone 23S
# http://www.processamentodigital.com.br/2013/07/27/lista-dos-codigos-epsg-mais-utilizados-no-brasil/

zone_shp = gpd.read_file('../data/sao-paulo/od/shapes/Distritos_2017_region.shp')
zone_shp.crs = {'init': 'epsg:31983'}  
zone_shp.to_crs(epsg='4326', inplace=True)

display(zone_shp)
# converting to km^2
zone_shp['Area_ha_2'] = zone_shp['Area_ha'] / 100
zone_shp['NumeroZona'] = zone_shp['NumeroDist']
zone_shp['NumeroMuni'] = 0

numeroMuni = []
for _, d in zone_shp.iterrows():
    if d['NomeDistri'] in distritos:
        numeroMuni.append(36)
    else:
        numeroMuni.append(0)
zone_shp['NumeroMuni'] = numeroMuni

protected_lanes = cinfra.load_protected_lanes('../data/sao-paulo/infraestrutura-cicloviaria/cet/Ciclovias.shp')
bike_lanes = cinfra.load_bike_lanes('../data/sao-paulo/infraestrutura-cicloviaria/cet/Ciclovias.shp')
sharrow_lanes = cinfra.load_sharrow_lanes('../data/sao-paulo/infraestrutura-cicloviaria/cet/Ciclorrotas.shp')

lanes = \
        gpd.read_file('../data/sao-paulo/geosampa/SIRGAS_SHP_logradouronbl/SIRGAS_SHP_logradouronbl.shp')
lanes.crs = {'init': 'epsg:31983'}  
lanes.to_crs(epsg='4326', inplace=True)


,NumeroDist,NomeDistri,Area_ha,geometry
0,1,Água Rasa,715.05,"POLYGON ((-46.55380 -23.56843, -46.55387 -23.5..."
1,2,Alto de Pinheiros,751.26,"POLYGON ((-46.69209 -23.54631, -46.69257 -23.5..."
2,3,Anhanguera,3339.95,"POLYGON ((-46.76082 -23.42733, -46.76088 -23.4..."
3,4,Aricanduva,686.26,"POLYGON ((-46.50153 -23.57941, -46.50154 -23.5..."
4,5,Artur Alvim,653.04,"POLYGON ((-46.47300 -23.54029, -46.47302 -23.5..."
...,...,...,...,...
129,130,São Caetano do Sul,1536.53,"POLYGON ((-46.58317 -23.64088, -46.58326 -23.6..."
130,131,São Lourenço da Serra,18674.34,"POLYGON ((-46.98852 -23.79984, -46.98847 -23.7..."
131,132,Suzano,20666.94,"POLYGON ((-46.38321 -23.61775, -46.38318 -23.6..."
132,133,Taboão da Serra,2043.37,"POLYGON ((-46.81819 -23.63701, -46.82138 -23.6..."


In [18]:
# Processing data per zone (may take some minutes)
zone_shp = geometry_intersection_length(zone_shp, 'NumeroZona', protected_lanes, 'protected_length')
zone_shp = geometry_intersection_length(zone_shp, 'NumeroZona', bike_lanes, 'bike_lane_length')
zone_shp = geometry_intersection_length(zone_shp, 'NumeroZona', sharrow_lanes, 'sharrow_length')
#zone_shp['sharrow_length'] = 0
zone_shp['lane_length'] = 0
#zone_shp = geometry_intersection_length(zone_shp, 'NumeroZona', lanes, 'lane_length')

In [24]:
### funcoes auxiliares   

colors = [('vermelho', 'red'), ('azul', 'blue'), ('verde', 'green'),  
          ('preto', 'black'), ('amarelo', 'gold'), ('roxo', 'purple'),
          ('vinho', 'crimson'), ('white', 'white')]
#colors = [('amarelo vermelho', 'YlOrRd'), ('amarelo marrom', 'YlOrBr'), ('vermelho', 'OrRd') ]

def show_choropleth_widgets(title, value_function, zones):
    # transform the lambda code in string, so we can pass as a string widget
    #pass the parameters as hidden widgets
    im = interact_manual(
        plot_bike_lanes_choropleth,
        geodf_zones = fixed(zones),
        value_function = fixed(value_function),
        title = fixed(title),
        color = widgets.Dropdown(options = colors, 
                                 description = 'Cor'),
        consider_protected = widgets.Checkbox(value=True, description = 'Considerar Ciclovias'),
        consider_bike_lane = widgets.Checkbox(value=True, description = 'Considerar Ciclofaixas'),
        consider_sharrow = widgets.Checkbox(value=True, description = 'Considerar Ciclorrotas'),
        plot_rmsp = widgets.Checkbox(value=True, description = 'Calcular RMSP')
    )
    im.widget.children[5].description = 'Gerar o mapa'

def plot_bike_lanes_choropleth(title, color, value_function, geodf_zones,
                               consider_protected = True, consider_bike_lane = True, consider_sharrow = True, 
                               plot_rmsp = False):
    geodf_zones['bike_length'] = 0
    if consider_protected:
        geodf_zones['bike_length'] += geodf_zones['protected_length']
    if consider_bike_lane:
        geodf_zones['bike_length'] += geodf_zones['bike_lane_length']
    if consider_sharrow:
        print('aqui')
        geodf_zones['bike_length'] += geodf_zones['sharrow_length']
    
    tooltip_columns = ['NomeDistri', 'Area_ha_2', 'lane_length', 'bike_length', 
                     'protected_length', 'bike_lane_length', 'sharrow_length']
    tooltip_aliases = ['Distrito', 'Área (km2)', 'Malha viária (km)', 'Malha cicloviária (km)', 
                      '..Ciclovias (km)', '..Ciclofaixas (km)', '..Ciclorrotas (km)']
    fmap = gr.map_around_sp(the_grid=None,zoom=10,plot_grid=False)
    spchoro.plot_choropleth(fmap, title, color, value_function, geodf_zones, 
                            tooltip_columns, tooltip_aliases)
    cinfra.plot_cycling_infra(fmap, protected_lanes, bike_lanes, sharrow_lanes, 'red', 'orange')
    
    fmap.zoom_control = False
    folium.LayerControl().add_to(fmap)
    file = 'maps/' + title.replace(' ', '_') + '.html'
    fmap.save(file)
    display(HTML('Salvo em <a href="' + file + '" target="_blank">' + file + '</a>'))
    display(fmap)

In [26]:
# Obtendo a km de ciclovias por zona
show_choropleth_widgets(title = 'São Paulo cycling infrastructure',
                        value_function = lambda x : x['bike_length'],
                        zones = zone_shp)

interactive(children=(Dropdown(description='Cor', options=(('vermelho', 'red'), ('azul', 'blue'), ('verde', 'g…

In [128]:
#km de ciclovia por área (densidade) por zona
show_choropleth_widgets('Cycling infrastructure density (km per km&sup2;)',
                        lambda x : x['bike_length'] / x['Area_ha_2'],
                        zones = zone_shp)

interactive(children=(Dropdown(description='Cor', options=(('amarelo vermelho', 'YlOrRd'), ('amarelo marrom', …

In [129]:
# Km de ciclovia por km da malha viária
show_choropleth_widgets('Lanes with cycling infrastructure (%)',
                        lambda x : 0 if x['lane_length'] == 0 else x['bike_length'] / x['lane_length'] * 100,
                        zones = zone_shp)

#show_choropleth_widgets('km de ciclovia por km de ruas',
#                        lambda x : 0 if x['lane_length'] == 0 else  \
#                        min(1, x['bike_length'] / x['lane_length']),
#                        zones = zone_shp)
                        

interactive(children=(Dropdown(description='Cor', options=(('amarelo vermelho', 'YlOrRd'), ('amarelo marrom', …

In [36]:
population = [84963,43117,65859,89622,105269,14383,69460,45057,33892,
              29265,264918,54196,157408,36948,62530,100713,216098,136623,
              275230,83281,85624,266681,203473,126597,211501,57365,109088,
              113615,142327,500787,103996,127662,106865,92570,278026,204871,
              223780,94609,24895,49863,281824,295434,135043,88692,267871,
              124122,185184,65739,69092,80229,107580,8258,83368,63133,46957,
              202321,17299,68258,158656,127820,111161,80187,65364,167931,93894,
              100164,56981,118459,247851,83717,118797,71560,84843,142347,158533,
              92081,151017,296042,130780,23651,37783,91672,197258,98438,127015,
              162486,94799,54331,167965,39485,113463,130484,104947,129919,104242,108441,]

zone_shp['population'] = population + [0]*38

# Km de ciclovia por km da malha viária
show_choropleth_widgets('Bike lanes (km) per capita',
                        lambda x : 0 if x['population'] == 0 else x['bike_length'] / x['population'],
                        zones = zone_shp)


interactive(children=(Dropdown(description='Cor', options=(('amarelo vermelho', 'YlOrRd'), ('amarelo marrom', …

In [14]:
#zone_shp_original = zone_shp.copy(True)
#zone_shp_aux = zone_shp.drop(labels=[178], axis=0)

#show_choropleth_widgets('km de ciclovia por km de ruas',
#                        lambda x : 0 if x['lane_length'] == 0 else x['bike_length'] / x['lane_length'],
#                        zones = zone_shp_aux)

In [130]:
# para salvar um csv

#columns = ['NomeDistri','Area_ha_2','population','lane_length',
#           'protected_length', 'bike_lane_length', 'sharrow_length', 
#           'density_area', 'density_percent']

#zone_shp['Area_ha_2'] = round(zone_shp['Area_ha_2'], 2)
#zone_shp['lane_length'] = round(zone_shp['lane_length'], 2)
#zone_shp['protected_length'] = round(zone_shp['protected_length'], 2)
#zone_shp['bike_lane_length'] = round(zone_shp['bike_lane_length'], 2)
#zone_shp['sharrow_length'] = round(zone_shp['sharrow_length'], 2)
#zone_shp['density_area'] = round(zone_shp['density_area'], 2)
#zone_shp['density_percent'] = round(zone_shp['density_percent'], 2)

#display(zone_shp.loc[zone_shp['NumeroMuni'] == 36][columns])

#zone_shp.loc[zone_shp['NumeroMuni'] == 36][columns].to_csv('districts_infra.csv')

In [193]:
trips = gpd.read_file('../data/sao-paulo/od/routes/bike_routes_all.shp')

zones_od = gpd.read_file('../data/sao-paulo/od/shapes/Zonas_2017_region.shp')
zones_od.crs = {'init': 'epsg:31983'}  
zones_od.to_crs(epsg='4326', inplace=True)

#display(trips)
#display(zone_shp)

trips_geometries = list(trips['geometry'])
trips_expansion = list(trips['FE_VIA'])

total_per_zone = [0]*134 #517 # hash to store trips indexed by zones

for z in range(len(zone_shp)):
    zone = zone_shp.iloc[z]['geometry']
    for i in range (len (trips_geometries)):
        if (trips_geometries[i].intersects(zone)):
            total_per_zone[z] += trips_expansion[i]
              

In [182]:
print(sum(total_per_zone))  

751455.3650559998


In [194]:
zone_shp['trips'] = total_per_zone
zone_shp['protected_length'] = 0
zone_shp['sharrow_length'] = 0
zone_shp['bike_lane_length'] = 0
zone_shp['bike_length'] = 0
zone_shp['lane_length'] = total_per_zone

# Km de ciclovia por km da malha viária
show_choropleth_widgets('Bike trips in São Paulo',
                        lambda x : x['trips'],
                        zones = zone_shp)


interactive(children=(Dropdown(description='Cor', options=(('amarelo vermelho', 'YlOrRd'), ('amarelo marrom', …

In [189]:
trips_origin = [0] * 134 #517
for _, t in trips.iterrows():
    #trips_origin[int(t['ZONA_D']) - 1] += t['FE_VIA']
    trips_origin[int(zones_od.iloc[int(t['ZONA_O']) - 1]['NumDistrit']) - 1] += t['FE_VIA']

zone_shp['origin_trips'] = trips_origin

print(sum(trips_origin))

383299.36855100014


In [192]:
zone_shp['lane_length'] = trips_origin
# Km de ciclovia por km da malha viária
show_choropleth_widgets('Bike trips in São Paulo',
                        lambda x : x['origin_trips'],
                        zones = zone_shp)

interactive(children=(Dropdown(description='Cor', options=(('amarelo vermelho', 'YlOrRd'), ('amarelo marrom', …

In [196]:
od_trips = pd.read_csv('../data/sao-paulo/od/OD_2017.csv')
od_trips.columns = [c[: c.index(',')] for c in od_trips.columns]
od_trips.set_index('ID_ORDEM', inplace=True, drop=False)

total_trips = [0]*134
bike_trips = [0]*134

for _, t in od_trips.iterrows():
    total_trips[int(zones_od.iloc[int(t['ZONA_O']) - 1]['NumDistrit']) - 1] += t['FE_VIA']
    if t['MODOPRIN'] == 15:
        bike_trips[int(zones_od.iloc[int(t['ZONA_O']) - 1]['NumDistrit']) - 1] += t['FE_VIA']
        
zone_shp['total_trips_od'] = total_trips
zone_shp['origin_trips_od'] = bike_trips

zone_shp['protected_length'] = zone_shp['total_trips_od']
zone_shp['sharrow_length'] = zone_shp['origin_trips_od']

In [200]:
print(sum(total_trips), sum(bike_trips))

# Km de ciclovia por km da malha viária
show_choropleth_widgets('Bike trips in São Paulo',
                        lambda x : x['origin_trips_od'],
                        zones = zone_shp)

# Km de ciclovia por km da malha viária
show_choropleth_widgets('Percentage of trips made by bicycle',
                        lambda x : 0 if x['total_trips_od'] == 0 else x['origin_trips_od'] / x['total_trips_od'] * 100,
                        zones = zone_shp)

42006659.86983397 376975.2340060002


interactive(children=(Dropdown(description='Cor', options=(('amarelo vermelho', 'YlOrRd'), ('amarelo marrom', …

interactive(children=(Dropdown(description='Cor', options=(('amarelo vermelho', 'YlOrRd'), ('amarelo marrom', …

In [201]:
zone_shp['bike_proportion'] = zone_shp['origin_trips_od'] / zone_shp['total_trips_od'] 

In [203]:
display(zone_shp[['NomeDistri', 'bike_proportion']].sort_values('bike_proportion'))

,NomeDistri,bike_proportion
133,Vargem Grande Paulista,0.000000
61,Perus,0.000000
58,Pedreira,0.000000
57,Parque do Carmo,0.000000
51,Marsilac,0.000000
...,...,...
42,Jardim Helena,0.040275
125,Santa Isabel,0.043526
87,Vila Guilherme,0.044993
98,Biritiba-Mirim,0.048985
